In [ ]:
import pandas as pd
import numpy as np
from glob import glob 

import cv2
import numpy as np

import pytesseract

"""
POC: Load Analyze-format MRI (.hdr/.img) and HDF5 data in Python
Requires: nibabel, h5py, numpy, matplotlib, pandas
"""

import os
import nibabel as nb
import matplotlib.pyplot as plt
from scipy import ndimage
import torch
from transformers import pipeline
from PIL import Image

2025-05-11 12:21:38.429056: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-11 12:21:38.582536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746980498.658082   94673 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746980498.675422   94673 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746980498.832489   94673 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

: 

# Loading the data

In [ ]:
# Load data
data_dir = "./data/"
data_l = glob(data_dir + '*.csv')
print(data_l)

event_df = pd.read_csv(data_l[0])
eye_track_df = pd.read_csv(data_l[1])
motion_df = pd.read_csv(data_l[2])

# Monkey K2 (Kuma)
# Electrodes [52 -> 64] and [121 -> 128] (inclusive) are in the medial wall of the macaque brain.
ecog_df = pd.read_csv(data_l[3])


['./data/Event.csv', './data/EyeTrack.csv', './data/Motion.csv', './data/ECoG.csv']


In [ ]:
image_path = "./imgs/K2.png"
raw_image = Image.open(image_path)

# Sulci Notes:
Sulci from right to left
1. Principal Sulcus (25 is at the principal sulcus)
2. Arcuate Sulcus
3. Central Sulcus
4. Intraparietal Sulcus
5. Lateral Sulcus or Sylvian Fissure (21 is at the conjunction of Superior Temporal Sulcus and Lateral Sulcus)
6. Superior Temporal Sulcus
7. Lunate Sulcus
8. Unknown. Possibly Calcarine Sulcus.